In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import math

torch.set_printoptions(sci_mode = False)

/home/kim/eccv/.conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [47]:
# grid discretisation over domain [0, 1] for x1 and x2
N_grid_x1 = 20 + 1 # 21 so steps are easy numbers
N_grid_x2 = N_grid_x1

N_grid = N_grid_x1 * N_grid_x2

x1 = np.linspace(0, 1, N_grid_x1) # x
x2 = np.linspace(0, 1, N_grid_x2) # y

# define distances
dx = 1 / (N_grid_x1 - 1)
dy = 1 / (N_grid_x2 - 1)

X1, X2 = np.meshgrid(x1, x2)

# To Do: normalise?

In [3]:
def simulate_convergence(X1, X2):
    U = X2
    V = X1
    return U, V

def simulate_merge(U, V):
    U = (X2 + 0.5)**2
    V = np.sin(X1 * math.pi)
    return U, V

def simulate_branching(U, V):
    U = X1 * X2
    V = - 0.5 * X2**2 + (X1 - 0.8)
    return U, V

def simulate_deflection(U, V):
    U = (X2 * 6 - 3)**2 + (X1 * 6 - 3)**2 + 3
    V = -2 * (X2 * 6 - 3) * (X1 * 6 - 3)
    return U, V

def simulate_ridge(U, V):
    U = X2 + 1
    V = - np.cos(3 * X1**3 * math.pi)
    return U, V

In [50]:
# flat [N, 1] arrays - unsqueeze for explicit last dim
X1_train = torch.tensor([0.8750, 0.1500, 0.2500, 0.5000, 0.3853, 0.9991, 0.4333, 0.1494, 0.6196, 0.7808, 0.5609, 0.5895, 0.3395, 0.7232]).unsqueeze(-1)

X2_train = torch.tensor([0.6750, 0.9750, 0.9000, 0.3500, 0.6010, 0.3451, 0.7451, 0.8499, 0.6240, 0.7218, 0.8389, 0.1664, 0.9771, 0.0598]).unsqueeze(-1)

X1_test = torch.tensor(X1.flatten()).squeeze().unsqueeze(-1)
X2_test = torch.tensor(X2.flatten()).squeeze().unsqueeze(-1)

In [11]:
sigma_n = torch.tensor([0.05])

In [21]:
U_train, V_train = simulate_convergence(X1_train, X2_train)

U_train_noisy = U_train + sigma_n * torch.randn_like(U_train)
V_train_noisy = V_train + sigma_n * torch.randn_like(V_train)

# Hyperparameters

In [51]:
X1_rows = X1_train_fixed
X2_rows = X2_train_fixed

X1_columns = X1_test
X2_columns = X2_test

In [55]:
X1_dist = (X1_rows.unsqueeze(1) - X1_columns.unsqueeze(0)).squeeze()
X2_dist = (X2_rows.unsqueeze(1) - X2_columns.unsqueeze(0)).squeeze()

X2_dist.shape

torch.Size([14, 441])

In [ ]:
lengthscale = torch.tensor([0.3])
output_scalar = torch.tensor([1.0])

def divergence_free_se_kernel(X1_rows,
                              X2_rows, 
                              X1_columns, 
                              X2_columns, 
                              sigma_f = output_scalar, 
                              l = lengthscale):
    
    # Calculate the (not quite Euclidean) distance between all pairs of points
    # This approach yields negative values as well
    # X1_dist: torch.Size([n_rows, n_columns])
    X1_dist = (X1_rows.unsqueeze(1) - X1_columns.unsqueeze(0)).squeeze()
    # ALTERNATIVE (pos.) torch.cdist(X1_rows, X1_columns): this is just torch.abs(X1_dist)

    # X2_dist: torch.Size([n_rows, n_columns])
    X2_dist = (X2_rows.unsqueeze(1) - X2_columns.unsqueeze(0)).squeeze()

    # torch.Size([n_rows, n_columns])
    upper_left = (1 - X2_dist.square().div(l**2)).div(l**2)

    # elementwise multiplication and division by scalar
    # Matlab version has negative values here! 
    upper_right = torch.mul(X1_dist, X2_dist).div(l**4)
    lower_left = upper_right
    lower_right = (1 - X1_dist.square().div(l**2)).div(l**2)

    # Concatenate upper and lower blocks column-wise, and then concatenate them row-wise
    # torch.Size([2 * n_train, 2 * n_test])
    block = torch.cat((torch.cat((upper_left, upper_right), 1), torch.cat((lower_left, lower_right), 1)), 0)

    # torch.Size([2 * n_train, 2 * n_test])
    # elementwise multiplication
    K = sigma_f.square() * block.mul((X1_dist.square() + X2_dist.square()).div(-2 * l**2).exp().tile(2, 2))

    return K